# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.126805e+02     1.575011e+00
 * time: 0.09175896644592285
     1     1.104842e+01     9.955929e-01
 * time: 1.3408207893371582
     2    -1.194877e+01     1.035576e+00
 * time: 1.4680168628692627
     3    -3.443861e+01     7.956738e-01
 * time: 1.636441946029663
     4    -4.809858e+01     5.934579e-01
 * time: 1.7884578704833984
     5    -5.698423e+01     2.146449e-01
 * time: 1.9259648323059082
     6    -5.974421e+01     1.299746e-01
 * time: 2.040303945541382
     7    -6.081511e+01     6.497854e-02
 * time: 2.147932767868042
     8    -6.123956e+01     8.137032e-02
 * time: 2.262969970703125
     9    -6.156762e+01     3.781047e-02
 * time: 2.370065927505493
    10    -6.179081e+01     3.476556e-02
 * time: 2.4841108322143555
    11    -6.197602e+01     2.468093e-02
 * time: 2.5971169471740723
    12    -6.205190e+01     1.927299e-02
 * time: 2.706157922744751
    13    -6.212692e+01     1.580293e-02
 * time: 2.819669961929321

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671065
    AtomicLocal         -18.8557682
    AtomicNonlocal      14.8522654
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485369 
    Xc                  -19.3336819

    total               -62.261666458623
